You have to work on the files:
*  [Books](https://github.com/gdv/foundationsCS/raw/master/progetti/2021/Books.csv.gz)
*  [Book ratings](https://github.com/gdv/foundationsCS/raw/master/progetti/2021/Book-Ratings.csv.gz)
*  [Users](https://github.com/gdv/foundationsCS/raw/master/progetti/2021/Users.csv.gz)
*  [Goodbooks books](https://github.com/gdv/foundationsCS/raw/master/progetti/2021/goodbooks.csv.gz)
*  [Goodbooks ratings](https://github.com/gdv/foundationsCS/raw/master/progetti/2021/goodbooks-ratings.csv.gz)

### Notes

1.    It is mandatory to use GitHub for developing the project.
1.    The project must be a jupyter notebook.
1.    There is no restriction on the libraries that can be used, nor on the Python version.
1.    To read those files, you need to use the `encoding = 'latin-1'` option.
1.    All questions on the project **must** be asked in a public channel on [Zulip](https://focs.zulipchat.com), otherwise no  answer will be given.

In [ ]:
import pandas as pd
import numpy as np
import re

In [ ]:
books = pd.read_csv("https://github.com/gdv/foundationsCS/raw/master/progetti/2021/Books.csv.gz", sep=';', encoding = 'latin-1')
bRatings = pd.read_csv("https://github.com/gdv/foundationsCS/raw/master/progetti/2021/Book-Ratings.csv.gz", sep=';', encoding = 'latin-1')
users = pd.read_csv("https://github.com/gdv/foundationsCS/raw/master/progetti/2021/Users.csv.gz", sep=';', encoding = 'latin-1')
goodBooks = pd.read_csv("https://github.com/gdv/foundationsCS/raw/master/progetti/2021/goodbooks.csv.gz", sep=',', encoding = 'latin-1')
gbRatings = pd.read_csv("https://github.com/gdv/foundationsCS/raw/master/progetti/2021/goodbooks-ratings.csv.gz", sep=',', encoding = 'latin-1')

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [ ]:
books.head()

In [ ]:
bRatings.head()

In [ ]:
users.head()

In [ ]:
goodBooks.head()

In [ ]:
gbRatings.head()

In [ ]:
goodBooks['original_publication_year']=goodBooks['original_publication_year'].astype(int)
goodBooks['isbn13']=goodBooks['isbn13'].astype(float).astype(int)

In [ ]:
users["Location"]

### 1. Normalize the location field of *Users* dataset, splitting into city, region, country.

In [ ]:
# sfruttiamo come è costruita la colonna Users, con separatore la virgola
occ = users["Location"].str.split(",", n = 2, expand = True)
users["City"]= occ[0]
users["Region"]= occ[1]
users["Country"]= occ[2]
users

In [ ]:
rows = users.Location.size
conta = 0
for line in users.Location:
  occ = re.finditer(',',line)
  count = 0
  for match in occ:
    count +=1
  if count!=2:
    conta +=1
print(f"percentuale diversi={round((conta/rows)*100,3)}%, numero={conta}, totale={rows}")

percentuale diversi=0.541%, numero=1510, totale=278858


### 2. For each book in the *Books* dataset, compute its average rating.

In [ ]:
joinBookRatings = pd.merge(books, bRatings, on='ISBN')
joinBookRatings.head()

In [ ]:
avgRatings = joinBookRatings.groupby(['ISBN','Book-Title','Book-Author','Year-Of-Publication','Publisher'], as_index=False)['Book-Rating'].mean()[['ISBN','Book-Title','Book-Author','Year-Of-Publication','Publisher','Book-Rating']]
avgRatings.rename({'Book-Rating': 'Media-Rating'}, axis=1, inplace=True)
avgRatings

### 3. For each book in the *GoodBooks* dataset, compute its average rating.

In [ ]:
goodBooks[['isbn', 'isbn13', 'authors', 'title', 'average_rating']].sort_values('average_rating',ascending=False)

,isbn,isbn13,authors,title,average_rating
24,545010225,9.780545e+12,"J.K. Rowling, Mary GrandPrÃ©",Harry Potter and the Deathly Hallows (Harry Po...,4.61
26,439785960,9.780440e+12,"J.K. Rowling, Mary GrandPrÃ©",Harry Potter and the Half-Blood Prince (Harry ...,4.54
23,439139600,9.780439e+12,"J.K. Rowling, Mary GrandPrÃ©",Harry Potter and the Goblet of Fire (Harry Pot...,4.53
17,043965548X,9.780440e+12,"J.K. Rowling, Mary GrandPrÃ©, Rufus Beck",Harry Potter and the Prisoner of Azkaban (Harr...,4.53
20,439358078,9.780439e+12,"J.K. Rowling, Mary GrandPrÃ©",Harry Potter and the Order of the Phoenix (Har...,4.46
...,...,...,...,...,...
33,1612130291,9.781612e+12,E.L. James,"Fifty Shades of Grey (Fifty Shades, #1)",3.67
27,140283331,9.780140e+12,William Golding,Lord of the Flies,3.64
2,316015849,9.780316e+12,Stephenie Meyer,"Twilight (Twilight, #1)",3.57
48,316160199,9.780316e+12,Stephenie Meyer,"New Moon (Twilight, #2)",3.52


In [ ]:
goodBooks['rating_calcolato'] = (goodBooks['ratings_1']*1+goodBooks['ratings_2']*2+goodBooks['ratings_3']*3+goodBooks['ratings_4']*4+goodBooks['ratings_5']*5)/goodBooks['work_ratings_count']
goodBooks[['isbn', 'isbn13', 'authors', 'title', 'average_rating','rating_calcolato']].sort_values('rating_calcolato',ascending=False)

,isbn,isbn13,authors,title,average_rating,rating_calcolato
24,545010225,9.780545e+12,"J.K. Rowling, Mary GrandPrÃ©",Harry Potter and the Deathly Hallows (Harry Po...,4.61,4.612756
26,439785960,9.780440e+12,"J.K. Rowling, Mary GrandPrÃ©",Harry Potter and the Half-Blood Prince (Harry ...,4.54,4.537725
17,043965548X,9.780440e+12,"J.K. Rowling, Mary GrandPrÃ©, Rufus Beck",Harry Potter and the Prisoner of Azkaban (Harr...,4.53,4.527866
23,439139600,9.780439e+12,"J.K. Rowling, Mary GrandPrÃ©",Harry Potter and the Goblet of Fire (Harry Pot...,4.53,4.525950
20,439358078,9.780439e+12,"J.K. Rowling, Mary GrandPrÃ©",Harry Potter and the Order of the Phoenix (Har...,4.46,4.463372
...,...,...,...,...,...,...
33,1612130291,9.781612e+12,E.L. James,"Fifty Shades of Grey (Fifty Shades, #1)",3.67,3.665074
27,140283331,9.780140e+12,William Golding,Lord of the Flies,3.64,3.642994
2,316015849,9.780316e+12,Stephenie Meyer,"Twilight (Twilight, #1)",3.57,3.571067
48,316160199,9.780316e+12,Stephenie Meyer,"New Moon (Twilight, #2)",3.52,3.521807


### 4. Merge together all rows sharing the same book title, author and publisher. We will call the resulting datset `merged books`. The books that have not been merged together will not appear in `merged books`.

In [ ]:
count = pd.DataFrame({'count'    : joinBookRatings.groupby( [ "Book-Title", "Book-Author", "Publisher"] ).size()}).reset_index()
merged_books = count[count['count'] >1]

In [ ]:
merged_books

### 5. For each book in `merged books` compute its average rating.

The average is computed considering all books in `books` that have been merged.

In [ ]:
merged_books['Media-Rating'] = joinBookRatings.groupby(['Book-Title','Book-Author','Publisher'], as_index=False)['Book-Rating'].mean()['Book-Rating']

In [ ]:
merged_books

### 6. For each book in `merged books` compute the minimum and maximum of the average ratings over all corresponding books in the `books` dataset.

Hence for each book in `merged books` we will have exactly two values (a minimum and a maximum)

In [ ]:
df1 = pd.DataFrame({'Media-Rating':joinBookRatings.groupby(['Book-Title','Book-Author','Publisher','ISBN'])['Book-Rating'].mean()}).reset_index()
merged_books['min'] = df1.groupby(['Book-Title','Book-Author','Publisher'], as_index=False)['Media-Rating'].min()['Media-Rating']
merged_books['max'] = df1.groupby(['Book-Title','Book-Author','Publisher'], as_index=False)['Media-Rating'].max()['Media-Rating']

In [ ]:
merged_books

### 7. For each book in `goodbooks`, compute the list of its authors. Assuming that the number of reviews with a text (column `work_text_reviews_count`) is split equally among all authors, find for each authors the total number of reviews with a text. We will call this quantity the *shared number of reviews with a text*.

In [ ]:
authors = goodBooks[goodBooks.duplicated(['authors'])==False]

In [ ]:
def remove(string):
    return " ".join(string.split())

In [ ]:
myDict= {}
reg = 'a-zA-Z.\s\-\u00A1-\u00FF'
re_enum = re.compile('(['+reg+']+),?', re.UNICODE)
temp5 = authors.set_index('authors')['work_text_reviews_count'].to_dict()
for key,value in temp5.items():
  occ = re_enum.findall(key)
  if occ:
    j=0
    for j in range(len(occ)):
      if remove(occ[j]) not in myDict:
        myDict[remove(occ[j])] = int(value/len(occ))
      else:
        myDict[remove(occ[j])] = myDict[remove(occ[j])] + int(value/len(occ))

In [ ]:
shared_number_of_reviews_with_text = pd.DataFrame(myDict.items(), columns=['Autori', 'shared_number_of_reviews_with_text'])

In [ ]:
shared_number_of_reviews_with_text

In [ ]:
#ATTENZIONE TUTTO IL CODICE SOPRA FUNZIONA MA 
#RIGA 'George Orwell, Erich Fromm, CelÃ¢l Ãster'
#viene divisa male in 
#['George Orwell', ' Erich Fromm', ' CelÃ¢l Ã', 'ster']

reg = 'a-zA-Z.\s\-\u00A1-\u00FF'
re_enum = re.compile('(['+reg+']+),?', re.UNICODE)
occ = re_enum.findall('George Orwell, Erich Fromm, CelÃ¢l Ãster')
print(occ)

### 8. For each year of publication, determine the author that has the largest value of the shared number of reviews with a text.

In [ ]:
myDict2= {}
reg = 'a-zA-Z.\s\-\u00A1-\u00FF'
re_enum = re.compile('(['+reg+']+),?', re.UNICODE)
temp6 = authors.set_index('authors')['original_publication_year'].to_dict()
for key,value in temp6.items():
  occ = re_enum.findall(key)
  if occ:
    j=0
    for j in range(len(occ)):
      if value not in myDict2:
        myDict2[value] = remove(occ[j])
      else:
        myDict2[value] = myDict2[value]+','+remove(occ[j])

In [ ]:
myDict3 = {}
for anno,aut in myDict2.items():
  for autore,review in myDict.items():
    if re.search(autore,aut):
      if anno not in myDict3:
        myDict3[anno] = {'autore':autore,'review':review}
      else:
        if myDict3[anno]['review'] <= review:
          myDict3[anno]['review'] = review
year_reviews =pd.DataFrame.from_dict(myDict3,orient='index')
year_reviews.index.name = 'anno'
year_reviews.reset_index(level=['anno'])

### 9. Assuming that there are no errors in the ISBN fields, find the books in both datasets, and compute the difference of average rating according to the ratings and the goodratings datasets

In [ ]:
libri = pd.DataFrame({'Rating_books':joinBookRatings.groupby(["ISBN"])['Book-Rating'].mean()}).reset_index()
good_libri = pd.DataFrame({'Rating_books':goodBooks.groupby(['isbn',"isbn13"])['average_rating'].mean()}).reset_index()
isbn_comuni = good_libri[good_libri.isbn.isin(libri.ISBN)]['isbn'].to_list()

In [ ]:
row_list = []
for lib in isbn_comuni:
  val1 = libri[libri['ISBN']== lib]['Rating_books'].values[0]
  val2 = good_libri[good_libri['isbn']==lib]['Rating_books'].values[0]
  dict1 = {
      'isbn'             : lib,
      'Rating_books'     : val1,
      'Rating_goodbooks' : val2,
      'Difference'       : abs(val2-val1)
  }
  row_list.append(dict1)
df4 = pd.DataFrame(row_list, columns=['isbn','Rating_books','Rating_goodbooks','Difference'])
df4

### 10. Split the users dataset according to the age. One dataset contains the users with unknown age, one with age 0-14, one with age 15-24, one with age 25-34, and so on.

In [ ]:
users[users['Age'] == users['Age'].max()]

In [ ]:
group_0_14 = users[users['Age']<15]
var_holder = {}
for i in range(1,24):
  var_holder["group_"+ str(i)+"5_"+str(i+1)+"4"] = users[(users['Age']>int(str(i)+"4")) & (users['Age']<int(str(i+1)+"5"))]
locals().update(var_holder)
#sono in dataframe con nome tipo group_25_34

### 11. Find the books that appear only in the goodbooks datasets.

In [ ]:
solo_in_goodbooks = goodBooks[~good_libri.isbn.isin(libri.ISBN)]

In [ ]:
solo_in_goodbooks

### 12. Assuming that each pair (author, title) identifies a book, for each book find the number of times it appears in the books dataset. Which books appear the most times?

In [ ]:
count_book = pd.DataFrame({'count': books.groupby( [ "Book-Title", "Book-Author"] ).size()}).reset_index()
count_book[count_book['count'] == count_book['count'].max()]

In [ ]:
count_book

### 13. Find the author with the highest average rating according to the goodbooks datasets.

In [ ]:
myDict5= {}
reg = 'a-zA-Z.\s\-\u00A1-\u00FF'
re_enum = re.compile('(['+reg+']+),?', re.UNICODE)
temp5 = authors.set_index('authors')['average_rating'].to_dict()
for key,value in temp5.items():
  occ = re_enum.findall(key)
  if occ:
    for j in range(len(occ)):
      if remove(occ[j]) not in myDict5:
        myDict5[remove(occ[j])] = {'rating':value,'num_occ':len(occ)}
      else:
        myDict5[remove(occ[j])] = {'rating':myDict5[remove(occ[j])]['rating']+value,'num_occ':len(occ)}

In [ ]:
author_max_rating = pd.DataFrame(myDict5.items(), columns=['Autori', 'rating_plus_count'])
normalized = pd.json_normalize(author_max_rating['rating_plus_count'])

author_max_rating = author_max_rating.join(normalized).drop(columns=['rating_plus_count'])
author_max_rating['rating']= author_max_rating['rating']/author_max_rating['num_occ']
author_max_rating.drop('num_occ', axis=1, inplace=True)
author_max_rating

In [ ]:
author_max_rating.loc[author_max_rating['rating'].idxmax()]